In [18]:
import os, sys
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
import sklearn.preprocessing as Preprocessing
import pickle
from datetime import datetime

sys.path.insert(0, 'src')
from utils.places365_pred_utils import get_class_category_dict, get_category_class_dict
from utils.utils import ensure_dir, write_lists, informal_log
from utils.attribute_utils import get_one_hot_attributes, get_frequent_attributes, hyperparam_search, partition_paths_by_congruency

ImportError: cannot import name 'partition_paths_by_congruency' from 'utils.attribute_utils' (/n/fs/ac-alignment/explain-alignment/src/utils/attribute_utils.py)

### Load features and attributes

In [2]:
# Load features
features_dir = os.path.join('saved', 'ADE20K', '0501_105640')
train_features_path = os.path.join(features_dir, 'train_features.pth')
val_features_path = os.path.join(features_dir, 'val_features.pth')
test_features_path = os.path.join(features_dir, 'test_features.pth')

train_features_dict = torch.load(train_features_path)
train_features = train_features_dict['features']
train_paths = train_features_dict['paths']

val_features_dict = torch.load(val_features_path)
val_features = val_features_dict['features']
val_paths = val_features_dict['paths']

test_features_dict = torch.load(test_features_path)
test_features = test_features_dict['features']
test_paths = test_features_dict['paths']

features = {
    'train': train_features,
    'val': val_features,
    'test': test_features
}
paths = {
    'train': train_paths,
    'val': val_paths,
    'test': test_paths
}
n_attributes = 1200
frequency_threshold = 150

# Load data and calculate attributes
data_path = os.path.join('data', 'ade20k', 'full_ade20k_imagelabels.pth')
data = torch.load(data_path)

print("Obtaining one hot encodings of attributes")
attributes = get_one_hot_attributes(
    data=data,
    paths=paths,
    n_attr=n_attributes
)
attribute_save_path = os.path.join(os.path.dirname(data_path), 'one_hot_attributes.pth')
if not os.path.exists(attribute_save_path):
    torch.save(attributes, attribute_save_path)
    print("Saved one hot attributes from train/val/test to {}".format(attribute_save_path))

print("Obtaining frequent attributes only")
freq_attributes, freq_attributes_one_hot = get_frequent_attributes(
    attributes=attributes,
    frequency_threshold=frequency_threshold
)

# Get indices of frequent attributes
frequent_attribute_idxs = np.nonzero(freq_attributes_one_hot)[0]
freq_attribute_save_path = os.path.join(os.path.dirname(data_path), 'frequency_filtered_one_hot_attributes.pth')
if not os.path.exists(freq_attribute_save_path):
    torch.save(freq_attributes, freq_attribute_save_path)
    print("Saved frequency filtered one hot attributes from train/val/test to {}".format(freq_attribute_save_path))

frequent_idx_save_path = os.path.join(os.path.dirname(data_path), 'frequent_attribute_idxs.pth')
if not os.path.exists(frequent_idx_save_path):
    torch.save(frequent_attribute_idxs, frequent_idx_save_path)
    print("Saved indices of frequent attributes from training to {}".format(frequent_idx_save_path))
# Load names of attributes
labels_path = os.path.join('data', 'broden1_224', 'label.csv')
attribute_label_dict = pd.read_csv(labels_path, index_col=0)['name'].to_dict()
print("Loaded human-readable labels)")

Obtaining one hot encodings of attributes
Processing attributes for train split


100%|██████████| 13326/13326 [00:00<00:00, 130250.36it/s]


Processing attributes for val split


100%|██████████| 4442/4442 [00:00<00:00, 157671.53it/s]


Processing attributes for test split


100%|██████████| 4442/4442 [00:00<00:00, 155199.66it/s]


Obtaining frequent attributes only
6 examples have no more attributes
8308/13326 examples affected
3 examples have no more attributes
2736/4442 examples affected
3 examples have no more attributes
2804/4442 examples affected
Loaded human-readable labels)


### For each attribute, create a linear classifier (hyperparameter search)

In [3]:
# def hyperparam_search(train_features,
#                                   train_labels, 
#                                   val_features, 
#                                   val_labels, 
#                                   regularization,
#                                   solver,
#                                   scaler=None,
#                                   Cs = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 3, 5],
#                                   log_path=None):
#     best_clf = None
#     best_acc = 0
    
#     if scaler is not None:
#         scaler.fit(train_features)
#         print("Scaler parameters: {}".format(scaler.get_params()))
#         train_features = scaler.transform(train_features)
#         val_features = scaler.transform(val_features)
#     for c in Cs:
#         clf = LogisticRegression(solver=solver, C=c, penalty=regularization)
#         clf.fit(train_features, train_labels)
#         score = clf.score(val_features, val_labels)
#         if score>best_acc:
#             best_acc = score
#             best_clf = clf
#             informal_log("Best accuracy: {} Regularization: {}".format(score, c), log_path)
    
#     return best_clf

In [3]:
cavs = {}
train_attributes = freq_attributes['train']
val_attributes = freq_attributes['val']

cavs_save_dir = os.path.join('saved', 'ADE20K', 'cav', datetime.now().strftime(r'%m%d_%H%M%S'))
if os.path.exists(cavs_save_dir):
    raise ValueError("Path {} already exists".format(cavs_save_dir))
ensure_dir(cavs_save_dir)

cavs_save_path = os.path.join(cavs_save_dir, 'cavs.pickle')
log_path = os.path.join(cavs_save_dir, 'log.txt')
n_frequent_attributes = len(frequent_attribute_idxs)

for idx, attribute_idx in tqdm(enumerate(frequent_attribute_idxs)):
    informal_log("[{}] {}/{} Calculating CAV for {}".format(
        datetime.now().strftime(r'%m%d_%H%M%S'),
        idx+1,
        n_frequent_attributes,
        attribute_label_dict[attribute_idx]), log_path)
    scaler = None # Preprocessing.StandardScaler()
    cav = hyperparam_search(
        train_features=train_features,
        train_labels=train_attributes[:, attribute_idx],
        val_features=val_features,
        val_labels=val_attributes[:, attribute_idx],
        scaler=scaler,
        solver='liblinear',
        regularization='l2',
        log_path=log_path
    )
    cavs[attribute_idx] = cav
    
    accuracy = cav.score(val_features, val_attributes[:, attribute_idx])
    informal_log("CAV accuracy for {} concept ({}): {:.4f}".format(
        attribute_label_dict[attribute_idx],
        attribute_idx,
        accuracy), log_path)
    # Save periodically
    if idx % 10 == 0:
        pickle.dump(cavs, open(cavs_save_path, 'wb'))
        
pickle.dump(cavs, open(cavs_save_path, 'wb'))


0it [00:00, ?it/s]

[0515_142919] 1/182 Calculating CAV for wall
Best accuracy: 0.8973435389464205 Regularization: 0.001
Best accuracy: 0.8975686627645205 Regularization: 0.005
Best accuracy: 0.8982440342188204 Regularization: 0.01
Best accuracy: 0.8991445294912201 Regularization: 0.05


1it [00:23, 23.72s/it]

CAV accuracy for wall concept (12): 0.8991
[0515_142942] 2/182 Calculating CAV for sky
Best accuracy: 0.9279603782080144 Regularization: 0.001
Best accuracy: 0.935164340387213 Regularization: 0.005
Best accuracy: 0.9371904547501125 Regularization: 0.01
Best accuracy: 0.9425934263845115 Regularization: 0.05


2it [00:45, 22.56s/it]

...

CAV accuracy for windowpane concept (15): 0.8109
[0515_143054] 5/182 Calculating CAV for tree
Best accuracy: 0.9040972534894192 Regularization: 0.001
Best accuracy: 0.909725348941918 Regularization: 0.005
Best accuracy: 0.9106258442143179 Regularization: 0.01


5it [02:01, 24.90s/it]

CAV accuracy for tree concept (16): 0.9106
[0515_143120] 6/182 Calculating CAV for building
Best accuracy: 0.9108509680324178 Regularization: 0.001
Best accuracy: 0.9153534443944169 Regularization: 0.005
Best accuracy: 0.9164790634849167 Regularization: 0.01


5it [02:07, 25.40s/it]


KeyboardInterrupt: 

In [4]:
'''
Have CAVs trained using ADE20K training set
For each split:

'''
cavs_path = 'saved/ADE20K/cav/scaled/0513_094421/cavs.pickle'
cavs_save_dir = os.path.dirname(cavs_path)
if 'scaled' in cavs_path:
    scale = True
else:
    scale = False
cavs = pickle.load(open(cavs_path, 'rb'))
splits = ['train', 'val', 'test']

if scale:
    import sklearn.preprocessing as Preprocessing
    scaler = Preprocessing.StandardScaler()
    scaler.fit(features['train'])
    scaled_features = {
        'train': scaler.transform(features['train']),
        'val': scaler.transform(features['val']),
        'test': scaler.transform(features['test'])
    }
concept_vectors_dict = {}
concept_vectors_save_path = os.path.join(cavs_save_dir, 'cav_attributes.pth')
if os.path.exists(concept_vectors_save_path):
    print("Concept presence vectors already exist at {}".format(concept_vectors_save_path))
else:
    for split in splits:
        if scale:
            split_features = scaled_features[split]
        else:
            split_features = features[split]
        n_samples = len(split_features)
        print("Obtaining concept presence vectors for {} split".format(split))

        concept_presence_vectors = []
        for attr_idx in tqdm(range(n_attributes)):
            if attr_idx in cavs:
                cav = cavs[attr_idx]
                concept_present = cav.predict(split_features)
                concept_presence_vectors.append(concept_present)
                assert len(concept_present) == n_samples
            else:
                concept_presence_vectors.append(np.zeros(n_samples))
        concept_presence_vectors = np.stack(concept_presence_vectors, axis=1)
        print(concept_presence_vectors.shape, n_samples)
        # Concept vectors only for frequent vectors; Turn it into one hot vector

        concept_vectors_dict[split] = concept_presence_vectors
        
    torch.save(concept_vectors_dict, concept_vectors_save_path)
    print("Saved concept present vectors from CAVs to {}".format(concept_vectors_save_path))


Concept presence vectors already exist at saved/ADE20K/cav/scaled/0513_094421/cav_attributes.pth


In [20]:
# Load cpvs
cav_attributes_path = 'saved/ADE20K/cav/scaled/0513_094421/cav_attributes.pth'
cav_attributes = torch.load(cav_attributes_path)

# Load model's predictions
# predictions_path = os.path.join(features_dir, '{}_logits_predictions.pth')
# train_predictions = torch.load(predictions_path.format('train'))['predictions']
# val_predictions = torch.load(predictions_path.format('val'))['predictions']
# test_predictions = torch.load(predictions_path.format('test'))['predictions']

# predictions = {
#     'train': train_predictions,
#     'val': val_predictions, 
#     'test': test_predictions
# }

prediction_path = os.path.join(
    'saved', 
    'PlacesCategoryClassification',
    '0510_102912',
    'ADE20K_predictions', 
    'saga', 
    '{}_outputs_predictions.pth')
splits = ['train', 'val', 'test']
predictions = {}
for split in splits:
    predictions[split] = torch.load(prediction_path.format(split))['predictions']




In [ ]:
# Train explainer
solver = 'saga'
penalty = 'l1'
c = 0.5
max_iter = 200
# hyperparam_search(
#     train_features=cav_attributes['train'],
#     train_labels=predictions['train'], 
#     val_features=cav_attributes['val'], 
#     val_labels=predictions['val'], 
#     regularization=penalty,
#     solver=solver,
#     scaler=None,
#     Cs = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 3, 5],
#     log_path=None)
explainer = LogisticRegression(
    solver=solver,
    penalty=penalty,
    C=c,
    multi_class='multinomial',
    max_iter=max_iter
)
explainer.fit(cav_attributes['train'], predictions['train'])
accuracy = explainer.score(cav_attributes['val'], predictions['val'])

In [19]:
def partition_paths_by_congruency(explainer_predictions,
                                  model_predictions,
                                  paths):
    '''
    Given list or arrays of explainer and model predictions, partition paths based on if predictions align

    Arg(s):
        explainer_predictions : N-length np.array
            predictions output by the explainer model
        model_predictions : N-length np.array
            predictions output by the model
        paths : N-length list
            paths of images corresponding to each data point

    Returns:
        dictionary : dict[str] : list
            key: 'congruent' or 'incongruent'
            value: list of paths
    '''
    n_samples = len(paths)
    assert len(explainer_predictions) == n_samples
    assert len(model_predictions) == n_samples, "Length of model predictions {} doesn't match n_samples {}".format(
        len(model_predictions), n_samples
    )

    incongruent_paths = []
    congruent_paths = []

    for explainer_prediction, model_prediction, path in tqdm(zip(
        explainer_predictions, model_predictions, paths
    )):
        if explainer_prediction == model_prediction:
            congruent_paths.append(path)
        else:
            incongruent_paths.append(path)

    return {
        'congruent': congruent_paths,
        'incongruent': incongruent_paths
    }

In [22]:

save_dir = os.path.dirname(prediction_path)

# Save CAV explainer
cav_explainer_dir = os.path.join(save_dir, 'cav_explainer')
ensure_dir(cav_explainer_dir)
cav_explainer_save_path = os.path.join(cav_explainer_dir, '{}_explainer_{}_{}.pickle'.format(solver, penalty, c))
if not os.path.exists(cav_explainer_save_path):
    pickle.dump(explainer, open(cav_explainer_save_path, 'wb'))
    print("Saved explainer trained on CAVs to {}".format(cav_explainer_save_path))
else:
    print("Explainer already exists at '{}'".format(cav_explainer_save_path))

# Save CAV explainer predictions
accuracy = explainer.score(cav_attributes['val'], predictions['val'])
print(accuracy)

explainer_outputs = explainer.decision_function(cav_attributes['val'])
explainer_probabilities = explainer.predict_proba(cav_attributes['val'])
explainer_predictions = explainer.predict(cav_attributes['val'])

validation_output = {
    'outputs': explainer_outputs,
    'probabilities': explainer_probabilities,
    'predictions': explainer_predictions
}
validation_output_path = os.path.join(cav_explainer_dir, '{}_explainer_{}_{}_validation.pth'.format(solver, penalty, c))
if not os.path.exists(validation_output_path):
    torch.save(validation_output, validation_output_path)
    print("Saved outputs from validation set to {}".format(validation_output_path))
else:
    print("Validation set outputs already saved to {}".format(validation_output_path))

# Save congruent/incongruent paths
congruency_paths = partition_paths_by_congruency(
    explainer_predictions=explainer_predictions,
    model_predictions=predictions['val'],
    paths=paths['val']
)
congruent_paths = congruency_paths['congruent']
incongruent_paths = congruency_paths['incongruent']
print(len(congruent_paths), len(incongruent_paths))
congruent_paths_path = os.path.join(save_dir, 'congruent_paths.txt')
incongruent_paths_path = os.path.join(save_dir, 'incongruent_paths.txt')
if not os.path.exists(congruent_paths_path) or not os.path.exists(incongruent_paths_path):
    write_lists(congruent_paths, congruent_paths_path)
    write_lists(incongruent_paths, incongruent_paths_path)
    print("Saved {} congruent paths to {} and {} incongruent paths to {}".format(
        len(congruent_paths),
        congruent_paths_path,
        len(incongruent_paths),
        incongruent_paths_path
    ))
else:
    print("Congruent paths already saved to {} and incongruent paths already saved to {}".format(
        congruent_paths_path, incongruent_paths_path
    ))

SyntaxError: incomplete input (530267873.py, line 53)